In [5]:
import datetime
import pywt
import pandas as pd
import glob
import numpy as np
import pytz

# def convert_unix_time_to_kst(unix_time):
#     kst = pytz.timezone('Asia/Seoul')
#     utc_time = datetime.datetime.fromtimestamp(unix_time / 1000000000.0, tz=pytz.utc)
#     kst_time = utc_time.astimezone(kst)
#     return kst_time

In [7]:
import csv
from datetime import datetime, timedelta

name_lt = ['choimingi', 'choimingi_Auto', 'houjonguk', 'houjonguk_Auto', 'jeongyubin', 'jeongyubin_Auto', 'leegahyeon', 'leegahyeon_Auto']
# course = ['A', 'B', 'C']
file_lt = ['Accelerometer', 'Gravity', 'Gyroscope', 'Location', 'Magnetometer', 'Orientation', 'TotalAcceleration']
# file_lt = ['Accelerometer', 'Gyroscope']


# time_interval = timedelta(seconds=1) # 1초 간격으로 데이터 추출
time_interval = timedelta(milliseconds=200) # 0.2초 간격으로 데이터 추출

for name in name_lt:
    path = f'../../Autopilot/Sensor/{name}/'
    path_lt = glob.glob(f'{path}*')
    for idx, i in enumerate(path_lt):
        for f in file_lt:                
            csv_path = f'{i}/{f}.csv' # 본인 경로에 맞게 수정
            output_path = f'{i}/200ms_{f}.csv'  # 본인 경로에 맞게 수정

            # sensor_csv = pd.read_csv(csv_path)

            with open(csv_path, 'r') as infile, open(output_path, 'w', newline='') as outfile:
                reader = csv.reader(infile)
                writer = csv.writer(outfile)

                # 헤더 복사
                header = next(reader)
                writer.writerow(header)

                previous_time = None

                for row in reader:
                    # UNIX 시간 열에서 값을 가져와서 정수로 변환
                    unix_time = int(row[0])
                    # UNIX 시간을 datetime 객체로 변환
                    # current_time = datetime.utcfromtimestamp(unix_time)
                    current_time = datetime.fromtimestamp(unix_time / 1000000000.0, tz=pytz.utc)

                    # 시작 시간 설정
                    if previous_time is None:
                        previous_time = current_time

                    # 1초 간격으로 데이터 추출 및 저장
                    while current_time >= previous_time:
                        writer.writerow(row)
                        previous_time += time_interval
                print(f'{name}_{c}_{idx+1}_{f}.csv --> done')
print('-----------------------------------------------------')

choimingi_C_1_Accelerometer.csv --> done
choimingi_C_1_Gravity.csv --> done
choimingi_C_1_Gyroscope.csv --> done
choimingi_C_1_Location.csv --> done
choimingi_C_1_Magnetometer.csv --> done
choimingi_C_1_Orientation.csv --> done
choimingi_C_1_TotalAcceleration.csv --> done
choimingi_C_2_Accelerometer.csv --> done
choimingi_C_2_Gravity.csv --> done
choimingi_C_2_Gyroscope.csv --> done
choimingi_C_2_Location.csv --> done
choimingi_C_2_Magnetometer.csv --> done
choimingi_C_2_Orientation.csv --> done
choimingi_C_2_TotalAcceleration.csv --> done
choimingi_Auto_C_1_Accelerometer.csv --> done
choimingi_Auto_C_1_Gravity.csv --> done
choimingi_Auto_C_1_Gyroscope.csv --> done
choimingi_Auto_C_1_Location.csv --> done
choimingi_Auto_C_1_Magnetometer.csv --> done
choimingi_Auto_C_1_Orientation.csv --> done
choimingi_Auto_C_1_TotalAcceleration.csv --> done
choimingi_Auto_C_2_Accelerometer.csv --> done
choimingi_Auto_C_2_Gravity.csv --> done
choimingi_Auto_C_2_Gyroscope.csv --> done
choimingi_Auto_C_2

## Normalization

In [639]:
name_lt = ['jeongyubin']
course = ['A', 'B', 'C']
file_lt = ['Accelerometer', 'Gyroscope']


def min_max_scaling(column):
    min_val = column.min()
    max_val = column.max()
    scaled_column = (column - min_val) / (max_val - min_val)
    return scaled_column


for name in name_lt:
    for c in course:
        path = f'./Sensor/{name}/{c}/'
        path_lt = glob.glob(f'{path}*')
        for i in path_lt:
            for f in file_lt:
                csv_path = f'{i}/pp_{f}.csv' # 본인 경로에 맞게 수정
                output_path = f'{i}/pp_nor_{f}.csv'  # 본인 경로에 맞게 수정
                
                df = pd.read_csv(csv_path)
                
                # 각 열을 Min-Max 정규화
                df['x'] = min_max_scaling(df['x'])
                df['y'] = min_max_scaling(df['y'])
                df['z'] = min_max_scaling(df['z'])
                
                df = df.round(4)
                df.to_csv(output_path, index=False)

## Time window segmentation & Wavelet Transform

In [29]:
features = pd.DataFrame()

In [32]:
df = pd.read_csv('../Sensor/jeongyubin/B/2023-08-15_02-02-36_jeongyubin_B_1/pp_nor_Accelerometer.csv')
# df = df.iloc[:, 2:5]
df

,time,seconds_elapsed,z,y,x
0,1692064956537911800,0.0729,0.6584,0.4995,0.3949
1,1692064957545910000,1.0809,0.6459,0.5052,0.3931
2,1692064958537917200,2.0729,0.6513,0.4931,0.4022
3,1692064959545918500,3.0809,0.6483,0.5373,0.3927
4,1692064960538001000,4.0730,0.6520,0.5341,0.3913
...,...,...,...,...,...
420,1692065376544614000,420.0796,0.6085,0.2747,0.5761
421,1692065377540032800,421.0750,0.6704,0.5072,0.3540
422,1692065378548120000,422.0831,0.6499,0.5862,0.3881
423,1692065379540056800,423.0751,0.6537,0.4970,0.3981


In [31]:
df = pd.read_csv('../Sensor/jeongyubin/B/2023-08-15_02-02-36_jeongyubin_B_1/pp_nor_Accelerometer.csv')
df.columns = ['aa' + col for col in df.columns]
df

,aatime,aaseconds_elapsed,aaz,aay,aax
0,1692064956537911800,0.0729,0.6584,0.4995,0.3949
1,1692064957545910000,1.0809,0.6459,0.5052,0.3931
2,1692064958537917200,2.0729,0.6513,0.4931,0.4022
3,1692064959545918500,3.0809,0.6483,0.5373,0.3927
4,1692064960538001000,4.0730,0.6520,0.5341,0.3913
...,...,...,...,...,...
420,1692065376544614000,420.0796,0.6085,0.2747,0.5761
421,1692065377540032800,421.0750,0.6704,0.5072,0.3540
422,1692065378548120000,422.0831,0.6499,0.5862,0.3881
423,1692065379540056800,423.0751,0.6537,0.4970,0.3981


In [33]:
# 데이터 이산 웨이블릿 변환
def wavelet_transform(data, level=2):
    # 웨이블릿 변환 수행 (레벨 1) - cA1 계산
    wavelet = 'db4'
    level = 1
    coeffs_level1 = pywt.wavedec(data, wavelet, level=level)
    cA1, cD1 = coeffs_level1[0], coeffs_level1[1]

    # 다음, 레벨 2로 웨이블릿 변환 수행 - cA2 계산
    level = 2
    coeffs_level2 = pywt.wavedec(data, wavelet, level=level)
    cA2, cD2 = coeffs_level2[0], coeffs_level2[1]
    return cA1, cA2, cD1, cD2

# 윈도우 사이즈와 스트라이드 설정
window_size = 60
stride = 1

# 데이터 프레임을 윈도우로 분할하고 Wavelet Packet Transform을 적용
level = 2
DWT_result = pd.DataFrame()

for i in range(0, len(df) - window_size + 1, stride):
    window = df.iloc[i:i+window_size]  # 60개 데이터 포인트로 이루어진 윈도우
    X = window['z']
    
    # Wavelet Packet Transform 수행
    cA1, cA2, cD1, cD2 = wavelet_transform(X.values, level=level)
    
    cA1 = np.around(cA1, 4)
    cA2 = np.around(cA2, 4)
    cD1 = np.around(cD1, 4)
    cD2 = np.around(cD2, 4)
    
    # 딕셔너리로 데이터를 추가
    new_data = {'cA1': [cA1], 'cA2': [cA2], 'cD1': [cD1], 'cD2': [cD2]}
    DWT_result = pd.concat([DWT_result, pd.DataFrame(new_data)], ignore_index=True)

# # 결과 DataFrame 출력
# result.to_csv('test_DWT2.csv', index=False)

In [11]:
# # Min-Max 정규화 함수 정의
# def min_max_normalize(data):
#     scaler = MinMaxScaler()
#     normalized_data = scaler.fit_transform(np.array(data).reshape(-1, 1)).flatten()
    
#     # 각 값들을 소수점 이하 2자리까지 저장
#     normalized_data = [round(value, 2) for value in normalized_data]
    
#     return normalized_data

# # DWT_result의 각 계수 컬럼에 Min-Max 정규화 및 소수점 이하 2자리까지 저장 적용
# DWT_result['coeffs_1_normalized'] = DWT_result['coeffs_1'].apply(min_max_normalize)
# DWT_result['coeffs_2_normalized'] = DWT_result['coeffs_2'].apply(min_max_normalize)
# DWT_result['coeffs_3_normalized'] = DWT_result['coeffs_3'].apply(min_max_normalize)

# # 필요 없어진 원본 계수 컬럼 삭제
# DWT_result = DWT_result.drop(columns=['coeffs_1', 'coeffs_2', 'coeffs_3'])

## Feature Extraction

- WE (Wavelet Energy)

In [34]:
# Wavelet energy 계산 함수
def calculate_wavelet_energy(coefficients):
    return np.sum(np.square(coefficients))

# 레벨, 계수별 Wavelet 에너지 계산 및 저장
DWT_result['cA1_energy'] = DWT_result['cA1'].apply(calculate_wavelet_energy)
DWT_result['cA2_energy'] = DWT_result['cA2'].apply(calculate_wavelet_energy)
DWT_result['cD1_energy'] = DWT_result['cD1'].apply(calculate_wavelet_energy)
DWT_result['cD2_energy'] = DWT_result['cD2'].apply(calculate_wavelet_energy)

In [35]:
# Define a function to calculate the WE Entropy
def calculate_we_entropy(energy_values):
    total_energy = sum(energy_values)  # Calculate the total energy

    # Calculate the probability distribution based on energy levels
    probabilities = [energy / total_energy for energy in energy_values]

    # Calculate Shannon's entropy
    entropy = -sum(p * np.log2(p) if p > 0 else 0 for p in probabilities)

    return entropy

DWT_result['WEE'] = DWT_result.apply(lambda row: calculate_we_entropy([row['cA1_energy'], row['cA2_energy'], row['cD1_energy'], row['cD2_energy']]), axis=1)

In [36]:
DWT_result

,cA1,cA2,cD1,cD2,cA1_energy,cA2_energy,cD1_energy,cD2_energy,WEE
0,"[0.9309, 0.9153, 0.929, 0.9183, 0.9274, 0.9208...","[1.3069, 1.302, 1.3133, 1.3062, 1.2969, 1.3361...","[0.0014, 0.0051, -0.0083, -0.0657, -0.0103, -0...","[0.0035, 0.0111, 0.0051, 0.0329, -0.0214, -0.0...",27.974077,34.008299,0.143566,0.046138,1.022457
1,"[0.943, 0.9201, 0.9162, 0.9104, 0.9414, 0.9263...","[1.3132, 1.2912, 1.3331, 1.3036, 1.2923, 1.356...","[-0.002, -0.0031, 0.038, 0.052, 0.0129, -0.008...","[0.0001, 0.0033, 0.02, 0.0591, 0.0641, 0.0979,...",28.264783,34.488632,0.086247,0.091669,1.020920
2,"[0.9354, 0.9288, 0.9199, 0.9274, 0.9208, 0.980...","[1.3196, 1.3051, 1.3232, 1.3136, 1.2983, 1.370...","[0.0028, -0.0078, -0.0658, -0.0103, -0.0354, 0...","[-0.0015, -0.0047, -0.0166, -0.0101, 0.0176, 0...",28.266424,34.539597,0.146155,0.063332,1.024587
3,"[0.947, 0.9457, 0.9096, 0.9414, 0.9263, 1.0097...","[1.3366, 1.3092, 1.3445, 1.3192, 1.3151, 1.379...","[0.004, 0.0364, 0.052, 0.0129, -0.0081, -0.021...","[-0.0068, -0.0233, -0.0249, -0.0732, -0.0975, ...",27.858498,33.308099,0.084932,0.395365,1.057529
4,"[0.9946, 0.9342, 0.9397, 0.9208, 0.9809, 0.963...","[1.3565, 1.3093, 1.4022, 1.3311, 1.339, 1.3662...","[-0.0179, -0.0632, -0.0109, -0.0354, 0.0528, -...","[0.0039, 0.0166, 0.0349, -0.0223, -0.0454, -0....",27.785887,33.509107,0.220292,0.091872,1.039022
...,...,...,...,...,...,...,...,...,...
361,"[0.9213, 0.897, 0.9041, 0.9042, 0.8939, 0.9835...","[1.2922, 1.2703, 1.2991, 1.2838, 1.2583, 1.376...","[0.0267, 0.0853, -0.0147, -0.0328, 0.0074, 0.0...","[0.0028, 0.007, -0.0269, -0.0077, -0.0031, 0.0...",28.202983,33.898210,0.147057,0.035698,1.022011
362,"[0.9594, 0.9382, 0.8392, 0.9033, 0.9419, 0.986...","[1.3226, 1.2406, 1.3724, 1.2712, 1.279, 1.3677...","[-0.0277, -0.0615, 0.058, -0.0032, -0.0182, -0...","[-0.0185, -0.0558, 0.0164, -0.0357, 0.0132, 0....",28.427713,34.332398,0.120800,0.068422,1.022970
363,"[1.0069, 0.8955, 0.98, 0.8939, 0.9835, 0.9678,...","[1.3497, 1.3053, 1.4025, 1.3319, 1.3343, 1.347...","[0.012, 0.0055, -0.0363, 0.0074, 0.0215, -0.00...","[0.0215, 0.0757, 0.0446, -0.0312, 0.0183, -0.0...",28.642253,34.831295,0.147009,0.041837,1.021630
364,"[0.9708, 0.9816, 0.8596, 0.9419, 0.9862, 0.936...","[1.3588, 1.2906, 1.3959, 1.3061, 1.3444, 1.323...","[0.012, 0.0401, -0.0012, -0.0182, -0.0051, 0.0...","[-0.0246, -0.0753, 0.0421, -0.0149, -0.003, -0...",28.538460,34.652443,0.110604,0.068928,1.021203


In [38]:
# features['A1D1_energy'] = DWT_result['A1D1_energy']
# features['A2D2_energy'] = DWT_result['A2D2_energy']
# features['WEE'] = DWT_result['WEE']

- Shannon’s Entropy

In [39]:
import math
def shannon_entropy(probabilities):
    """
    Calculate Shannon entropy for a given probability distribution.

    :param probabilities: A list of probabilities for each possible outcome.
    :return: Shannon entropy as a floating-point number.
    """
    entropy = 0
    for p in probabilities:
        if p > 0:
            entropy += p * math.log2(1 / p)
    return entropy

In [40]:
# def shannon_entropy(data):
#     unique, counts = np.unique(data, return_counts=True)
#     probabilities = counts / len(data)
#     entropy = -np.sum(probabilities * np.log2(probabilities))
#     return entropy

window_size = 60
stride = 1
entropy_result = pd.DataFrame(columns=['Raw_entropy', 'cA_entropy', 'cD_entropy'])

for i in range(0, len(df) - window_size + 1, stride):
    window = df.iloc[i:i+window_size]  # 60개 데이터 포인트로 이루어진 윈도우
    X = window['z']
    # Shannon 엔트로피 계산
    entropy = shannon_entropy(X.values)
    entropy_cA = shannon_entropy(np.concatenate((DWT_result['cA1'][i], DWT_result['cA2'][i])))
    entropy_cD = shannon_entropy(np.concatenate((DWT_result['cD1'][i], DWT_result['cD2'][i])))

    new_data = {'Raw_entropy': [entropy], 'cA_entropy': [entropy_cA], 'cD_entropy': [entropy_cD]}
    entropy_result = pd.concat([entropy_result, pd.DataFrame(new_data)], ignore_index=True)

In [41]:
entropy_result

,Raw_entropy,cA_entropy,cD_entropy
0,23.966090,-6.372774,4.409187
1,23.858269,-6.917610,6.032180
2,23.801011,-6.952894,4.891373
3,23.928414,-5.825655,3.814536
4,24.014327,-5.882650,5.553637
...,...,...,...
361,23.779699,-6.450466,4.415483
362,23.768962,-6.917879,4.469757
363,23.715440,-7.420066,4.298557
364,23.753469,-7.221691,4.192904


In [42]:
features['Raw_entropy'] = entropy_result['Raw_entropy']
features['cA_entropy'] = entropy_result['cA_entropy']
features['cD_entropy'] = entropy_result['cD_entropy']

- The k Percentile

In [43]:
### Raw percentile ###
Raw_percentile_result = pd.DataFrame(columns=['Raw_5_percentile', 'Raw_25_percentile', 'Raw_75_percentile', 'Raw_95_percentile'])

def percentile(data):
    percentile_5 = np.percentile(data, 5)
    percentile_25 = np.percentile(data, 25)
    percentile_75 = np.percentile(data, 75)
    percentile_95 = np.percentile(data, 95)
    return percentile_5, percentile_25, percentile_75, percentile_95

for i in range(0, len(df) - window_size + 1, stride):
    window = df.iloc[i:i+window_size]  # 60개 데이터 포인트로 이루어진 윈도우
    X = window['z']
    # print(X.values)
    
    percent_5, percent_25, percent_75, percent_95 = percentile(X.values)
    
    new_data = {'Raw_5_percentile': [percent_5], 'Raw_25_percentile': [percent_25], 'Raw_75_percentile': [percent_75], 'Raw_95_percentile': [percent_95]}
    Raw_percentile_result = pd.concat([Raw_percentile_result, pd.DataFrame(new_data)], ignore_index=True)

In [44]:
Raw_percentile_result

,Raw_5_percentile,Raw_25_percentile,Raw_75_percentile,Raw_95_percentile
0,0.559050,0.627275,0.689700,0.728635
1,0.559050,0.627275,0.693525,0.745830
2,0.559050,0.627275,0.694225,0.745830
3,0.554025,0.624675,0.694225,0.745830
4,0.516500,0.623250,0.694225,0.745830
...,...,...,...,...
361,0.576390,0.629600,0.687450,0.737520
362,0.576390,0.629600,0.687450,0.737520
363,0.576390,0.631900,0.687450,0.737520
364,0.576390,0.631900,0.686350,0.737520


In [45]:
### cA percentile ###
cA_percentile_result = pd.DataFrame(columns=['cA_5_percentile', 'cA_25_percentile', 'cA_75_percentile', 'cA_95_percentile'])

for i in range(len(DWT_result)):
    X = np.concatenate((DWT_result['cA1'][i], DWT_result['cA2'][i]))
    
    percent_5, percent_25, percent_75, percent_95 = percentile(X)
    
    new_data = {'cA_5_percentile': [percent_5], 'cA_25_percentile': [percent_25], 'cA_75_percentile': [percent_75], 'cA_95_percentile': [percent_95]}
    cA_percentile_result = pd.concat([cA_percentile_result, pd.DataFrame(new_data)], ignore_index=True)

In [46]:
### cD percentile ###
cD_percentile_result =  pd.DataFrame(columns=['cD_5_percentile', 'cD_25_percentile', 'cD_75_percentile', 'cD_95_percentile'])

for i in range(len(DWT_result)):
    X = np.concatenate((DWT_result['cD1'][i], DWT_result['cD2'][i]))
    
    percent_5, percent_25, percent_75, percent_95 = percentile(X)
    
    new_data = {'cD_5_percentile': [percent_5], 'cD_25_percentile': [percent_25], 'cD_75_percentile': [percent_75], 'cD_95_percentile': [percent_95]}
    cD_percentile_result = pd.concat([cD_percentile_result, pd.DataFrame(new_data)], ignore_index=True)

In [47]:
percentile_result = pd.concat([Raw_percentile_result, cA_percentile_result, cD_percentile_result], axis=1)
percentile_result

,Raw_5_percentile,Raw_25_percentile,Raw_75_percentile,Raw_95_percentile,cA_5_percentile,cA_25_percentile,cA_75_percentile,cA_95_percentile,cD_5_percentile,cD_25_percentile,cD_75_percentile,cD_95_percentile
0,0.559050,0.627275,0.689700,0.728635,0.86592,0.9183,1.2969,1.36104,-0.10294,-0.0291,0.0322,0.09204
1,0.559050,0.627275,0.693525,0.745830,0.83750,0.9125,1.2912,1.40074,-0.08518,-0.0190,0.0518,0.10146
2,0.559050,0.627275,0.694225,0.745830,0.86592,0.9199,1.2983,1.37894,-0.09346,-0.0292,0.0347,0.11008
3,0.554025,0.624675,0.694225,0.745830,0.82150,0.9110,1.2622,1.36640,-0.12810,-0.0545,0.0305,0.07210
4,0.516500,0.623250,0.694225,0.745830,0.83948,0.9063,1.2809,1.40660,-0.14228,-0.0354,0.0351,0.12556
...,...,...,...,...,...,...,...,...,...,...,...,...
361,0.576390,0.629600,0.687450,0.737520,0.84570,0.9130,1.2703,1.36496,-0.07622,-0.0147,0.0303,0.07978
362,0.576390,0.629600,0.687450,0.737520,0.84808,0.9348,1.2900,1.36798,-0.08022,-0.0357,0.0216,0.11316
363,0.576390,0.631900,0.687450,0.737520,0.86788,0.9194,1.3013,1.35530,-0.07944,-0.0280,0.0303,0.07124
364,0.576390,0.631900,0.686350,0.737520,0.85736,0.9281,1.2898,1.37614,-0.09122,-0.0246,0.0216,0.10856


- Mean~Kurtosis

In [48]:
### original Mean~Kurtosis ###
from scipy.stats import skew, kurtosis

# 윈도우 크기 및 스트라이드 설정
window_size = 60
stride = 1

# 결과 데이터프레임 초기화
Raw_etc_result = pd.DataFrame(columns=['Raw_Mean', 'Raw_Median', 'Raw_Variance', 'Raw_Std_deviation', 'Raw_RMS', 'Raw_Skewness', 'Raw_Kurtosis'])

for i in range(0, len(df) - window_size + 1, stride):
    window = df.iloc[i:i + window_size]
    X = window['z']

    # 특징 추출
    mean = X.mean()
    median = X.median()
    variance = X.var()
    std_dev = X.std()
    rms = np.sqrt((X ** 2).mean())
    skewness = skew(X)
    kurt = kurtosis(X)

    Raw_etc_result = pd.concat([Raw_etc_result, pd.DataFrame({
        'Raw_Mean': [mean],
        'Raw_Median': [median],
        'Raw_Variance': [variance],
        'Raw_Std_deviation': [std_dev],
        'Raw_RMS': [rms],
        'Raw_Skewness': [skewness],
        'Raw_Kurtosis': [kurt]})], ignore_index=True)

In [50]:
### cA Mean~Kurtosis ###

# 결과 데이터프레임 초기화
cA_etc_result = pd.DataFrame(columns=['cA_Mean', 'cA_Median', 'cA_Variance', 'cA_Std_deviation', 'cA_RMS', 'cA_Skewness', 'cA_Kurtosis'])

for i in range(len(DWT_result)):
    X = np.concatenate((DWT_result['cA1'][i], DWT_result['cA2'][i]))
    
    # 특징 추출
    mean = X.mean()
    median = np.median(X)
    variance = X.var()
    std_dev = X.std()
    rms = np.sqrt((X**2).mean())
    skewness = skew(X)
    kurt = kurtosis(X)

    # 결과 데이터프레임에 특징 추가
    cA_etc_result = pd.concat([cA_etc_result, pd.DataFrame({
                            'cA_Mean': [mean],
                            'cA_Median': [median],
                            'cA_Variance': [variance],
                            'cA_Std_deviation': [std_dev],
                            'cA_RMS': [rms],
                            'cA_Skewness': [skewness],
                            'cA_Kurtosis': [kurt]})], ignore_index=True)

In [51]:
### cD Mean~Kurtosis ###

# 결과 데이터프레임 초기화
cD_etc_result = pd.DataFrame(columns=['cD_Mean', 'cD_Median', 'cD_Variance', 'cD_Std_deviation', 'cD_RMS', 'cD_Skewness', 'cD_Kurtosis'])

for i in range(len(DWT_result)):
    X = np.concatenate((DWT_result['cD1'][i], DWT_result['cD2'][i]))
    
    # 특징 추출
    mean = X.mean()
    median = np.median(X)
    variance = X.var()
    std_dev = X.std()
    rms = np.sqrt((X**2).mean())
    skewness = skew(X)
    kurt = kurtosis(X)

    # 결과 데이터프레임에 특징 추가
    cD_etc_result = pd.concat([cD_etc_result, pd.DataFrame({
                            'cD_Mean': [mean],
                            'cD_Median': [median],
                            'cD_Variance': [variance],
                            'cD_Std_deviation': [std_dev],
                            'cD_RMS': [rms],
                            'cD_Skewness': [skewness],
                            'cD_Kurtosis': [kurt]})], ignore_index=True)

In [52]:
### cD2 Mean~Kurtosis ###

# 결과 데이터프레임 초기화
etc_result4 = pd.DataFrame(columns=['cD2_Mean', 'cD2_Median', 'cD2_Variance', 'cD2_Std_deviation', 'cD2_RMS', 'cD2_Skewness', 'cD2_Kurtosis'])

for i in range(len(DWT_result)):
    X = DWT_result['cD2'][i]
    
    # 특징 추출
    mean = X.mean()
    median = np.median(X)
    variance = X.var()
    std_dev = X.std()
    rms = np.sqrt((X**2).mean())
    skewness = skew(X)
    kurt = kurtosis(X)

    # 결과 데이터프레임에 특징 추가
    etc_result4 = pd.concat([etc_result4, pd.DataFrame({
                            'cD2_Mean': [mean],
                            'cD2_Median': [median],
                            'cD2_Variance': [variance],
                            'cD2_Std_deviation': [std_dev],
                            'cD2_RMS': [rms],
                            'cD2_Skewness': [skewness],
                            'cD2_Kurtosis': [kurt]})], ignore_index=True)

In [53]:
etc_result = pd.concat([Raw_etc_result, cA_etc_result, cD_etc_result], axis=1)
etc_result

,Raw_Mean,Raw_Median,Raw_Variance,Raw_Std_deviation,Raw_RMS,Raw_Skewness,Raw_Kurtosis,cA_Mean,cA_Median,cA_Variance,...,cA_RMS,cA_Skewness,cA_Kurtosis,cD_Mean,cD_Median,cD_Variance,cD_Std_deviation,cD_RMS,cD_Skewness,cD_Kurtosis
0,0.648437,0.65005,0.004880,0.069857,0.652126,-1.713497,7.538297,1.062908,0.9633,0.039706,...,1.081424,0.336830,-1.114894,-0.001781,0.0035,0.003576,0.059801,0.059827,-0.035767,1.289171
1,0.650315,0.65005,0.005130,0.071622,0.654182,-1.583438,6.848714,1.068225,0.9869,0.042923,...,1.088130,0.289154,-1.066139,0.013223,0.0076,0.003182,0.056410,0.057939,0.053956,0.083699
2,0.651397,0.65165,0.005190,0.072043,0.655302,-1.591145,6.724048,1.069462,0.9712,0.041270,...,1.088586,0.336751,-1.100051,0.004328,-0.0004,0.003934,0.062720,0.062870,0.597609,1.110352
3,0.646923,0.65040,0.006392,0.079948,0.651763,-1.642253,5.214664,1.054025,0.9869,0.043119,...,1.074284,0.056547,-0.668343,-0.015428,-0.0079,0.008824,0.093937,0.095196,0.851338,7.620569
4,0.644762,0.65040,0.006666,0.081646,0.649825,-1.528175,4.490184,1.055538,0.9712,0.042349,...,1.075411,0.346104,-0.899504,-0.002217,0.0039,0.005885,0.076714,0.076746,-0.286026,0.242450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,0.654832,0.65660,0.003229,0.056827,0.657252,-1.086176,4.238149,1.065808,0.9708,0.035775,...,1.082460,0.475283,-1.492298,0.001666,0.0015,0.003445,0.058698,0.058721,-1.474136,5.783790
362,0.655042,0.65660,0.003233,0.056862,0.657464,-1.095029,4.236963,1.071109,0.9594,0.036878,...,1.088188,0.437346,-1.503840,0.002438,0.0042,0.003564,0.059702,0.059751,0.997583,1.902316
363,0.656252,0.65660,0.003130,0.055944,0.658592,-1.169373,4.758308,1.077117,0.9715,0.037433,...,1.094356,0.458148,-1.562156,-0.003091,-0.0015,0.003554,0.059612,0.059692,-1.288642,4.904430
364,0.655528,0.65605,0.003101,0.055687,0.657850,-1.153095,4.836155,1.074606,0.9708,0.037504,...,1.091916,0.434074,-1.529497,0.000919,0.0042,0.003387,0.058194,0.058201,0.990042,2.405425


In [687]:
features

,A1D1_energy,A2D2_energy,WEE,orizinal_entropy,cA1_entropy,cA2_entropy,cD1_entropy,cD2_entropy
0,27.686288,31.149462,0.997499,24.875515,4.908510,-8.088996,3.951457,1.838479
1,25.228632,31.355733,0.991526,24.862751,4.854788,-8.247846,2.418375,1.816586
2,27.916939,32.074378,0.996533,24.826024,4.821596,-8.344118,3.999195,1.990965
3,25.258227,32.315169,0.989135,24.781981,4.782353,-8.494734,2.378887,1.875455
4,27.647997,31.909666,0.996303,24.761277,4.769470,-8.494714,3.522075,1.974006
...,...,...,...,...,...,...,...,...
552,28.305330,34.580994,0.992804,24.527611,3.663732,-10.643265,2.737070,1.422497
553,27.689082,34.388985,0.991581,24.559595,3.724517,-10.598345,2.382061,1.260590
554,27.932589,35.284772,0.990221,24.578449,3.782127,-10.500461,2.506340,1.735253
555,26.978382,33.504319,0.991586,24.802881,4.305794,-9.484058,2.404188,1.539252


In [562]:
features = pd.concat([features, percentile_result], axis=1)
features

,A1D1_energy,A2D2_energy,WEE,orizinal_entropy,cA1_entropy,cA2_entropy,cD1_entropy,cD2_entropy,orizinal_percentile_5,orizinal_percentile_25,...,cA2_percentile_75,cA2_percentile_95,cD1_percentile_5,cD1_percentile_25,cD1_percentile_75,cD1_percentile_95,cD2_percentile_5,cD2_percentile_25,cD2_percentile_75,cD2_percentile_95
0,14.372492,16.007046,0.997911,5.873557,5.044394,4.321928,5.044394,4.321928,0.368130,0.424050,...,0.6749,0.72984,0.55752,0.6084,0.6749,0.72984,0.55752,0.6084,0.6749,0.72984
1,13.341156,15.972023,0.994182,5.873557,5.044394,4.321928,5.044394,4.321928,0.368130,0.424050,...,0.6648,0.74068,0.56600,0.6153,0.6648,0.74068,0.56600,0.6153,0.6648,0.74068
2,14.664167,17.180505,0.995491,5.873557,5.044394,4.321928,5.044394,4.221928,0.368130,0.424050,...,0.6778,0.74136,0.56264,0.6053,0.6778,0.74136,0.56264,0.6053,0.6778,0.74136
3,13.469172,17.691419,0.986715,5.873557,5.044394,4.321928,5.044394,4.321928,0.368130,0.424375,...,0.6716,0.74068,0.56644,0.6154,0.6716,0.74068,0.56644,0.6154,0.6716,0.74068
4,14.749480,16.520934,0.997684,5.873557,5.044394,4.321928,5.044394,4.321928,0.368130,0.424375,...,0.6778,0.72984,0.56600,0.6086,0.6778,0.72984,0.56600,0.6086,0.6778,0.72984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516,14.737896,18.363848,0.991327,5.873557,5.044394,4.221928,5.044394,4.321928,0.414535,0.442375,...,0.6915,0.78768,0.61054,0.6347,0.6915,0.78768,0.61054,0.6347,0.6915,0.78768
517,15.215484,17.739707,0.995764,5.873557,5.044394,4.321928,5.044394,4.321928,0.414535,0.442375,...,0.6717,0.77112,0.61828,0.6329,0.6717,0.77112,0.61828,0.6329,0.6717,0.77112
518,14.169982,16.940546,0.994271,5.873557,5.044394,4.321928,5.044394,4.321928,0.414535,0.442375,...,0.6905,0.75484,0.61244,0.6347,0.6905,0.75484,0.61244,0.6347,0.6905,0.75484
519,15.302014,17.730795,0.996097,5.873557,5.044394,4.321928,5.044394,4.321928,0.416135,0.443450,...,0.6785,0.77112,0.62258,0.6385,0.6785,0.77112,0.62258,0.6385,0.6785,0.77112


In [688]:
features = pd.concat([features, etc_result], axis=1)
features

,A1D1_energy,A2D2_energy,WEE,orizinal_entropy,cA1_entropy,cA2_entropy,cD1_entropy,cD2_entropy,original_Mean,original_Median,...,cD1_RMS,cD1_Skewness,cD1_Kurtosis,cD2_Mean,cD2_Median,cD2_Variance,cD2_Std_deviation,cD2_RMS,cD2_Skewness,cD2_Kurtosis
0,27.686288,31.149462,0.997499,24.875515,4.908510,-8.088996,3.951457,1.838479,0.628808,0.62510,...,0.086153,1.192917,3.837308,-0.005990,0.00245,0.004921,0.070151,0.070406,-0.756257,0.502008
1,25.228632,31.355733,0.991526,24.862751,4.854788,-8.247846,2.418375,1.816586,0.629080,0.62540,...,0.090572,-1.380191,3.455281,-0.006875,-0.00470,0.006113,0.078183,0.078485,-0.365148,0.211409
2,27.916939,32.074378,0.996533,24.826024,4.821596,-8.344118,3.999195,1.990965,0.629827,0.62540,...,0.086262,1.194736,3.890219,0.005100,0.00085,0.005647,0.075147,0.075320,0.944897,1.018330
3,25.258227,32.315169,0.989135,24.781981,4.782353,-8.494734,2.378887,1.875455,0.630710,0.62540,...,0.090534,-1.390647,3.477314,0.005830,-0.00195,0.004833,0.069516,0.069760,-0.035377,1.011914
4,27.647997,31.909666,0.996303,24.761277,4.769470,-8.494714,3.522075,1.974006,0.631172,0.62650,...,0.081614,1.400508,5.250005,-0.000870,0.00990,0.004515,0.067194,0.067200,-0.897766,1.184709
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
552,28.305330,34.580994,0.992804,24.527611,3.663732,-10.643265,2.737070,1.422497,0.638672,0.62525,...,0.046166,0.541381,0.302102,-0.010755,-0.01960,0.003617,0.060146,0.061100,0.473889,0.887487
553,27.689082,34.388985,0.991581,24.559595,3.724517,-10.598345,2.382061,1.260590,0.638003,0.62390,...,0.048420,-0.256671,0.522196,-0.009470,0.00790,0.003039,0.055126,0.055934,-2.124340,5.710925
554,27.932589,35.284772,0.990221,24.578449,3.782127,-10.500461,2.506340,1.735253,0.637607,0.62390,...,0.043097,0.541286,0.742225,0.006430,0.00675,0.002085,0.045667,0.046118,-0.517669,0.670844
555,26.978382,33.504319,0.991586,24.802881,4.305794,-9.484058,2.404188,1.539252,0.633807,0.62340,...,0.047878,-0.321683,0.667451,-0.000010,0.00245,0.002417,0.049165,0.049165,-0.270820,0.446882


In [694]:
features['label'] = 0
features

,A1D1_energy,A2D2_energy,WEE,orizinal_entropy,cA1_entropy,cA2_entropy,cD1_entropy,cD2_entropy,original_Mean,original_Median,...,cD1_Skewness,cD1_Kurtosis,cD2_Mean,cD2_Median,cD2_Variance,cD2_Std_deviation,cD2_RMS,cD2_Skewness,cD2_Kurtosis,label
0,27.686288,31.149462,0.997499,24.875515,4.908510,-8.088996,3.951457,1.838479,0.628808,0.62510,...,1.192917,3.837308,-0.005990,0.00245,0.004921,0.070151,0.070406,-0.756257,0.502008,0
1,25.228632,31.355733,0.991526,24.862751,4.854788,-8.247846,2.418375,1.816586,0.629080,0.62540,...,-1.380191,3.455281,-0.006875,-0.00470,0.006113,0.078183,0.078485,-0.365148,0.211409,0
2,27.916939,32.074378,0.996533,24.826024,4.821596,-8.344118,3.999195,1.990965,0.629827,0.62540,...,1.194736,3.890219,0.005100,0.00085,0.005647,0.075147,0.075320,0.944897,1.018330,0
3,25.258227,32.315169,0.989135,24.781981,4.782353,-8.494734,2.378887,1.875455,0.630710,0.62540,...,-1.390647,3.477314,0.005830,-0.00195,0.004833,0.069516,0.069760,-0.035377,1.011914,0
4,27.647997,31.909666,0.996303,24.761277,4.769470,-8.494714,3.522075,1.974006,0.631172,0.62650,...,1.400508,5.250005,-0.000870,0.00990,0.004515,0.067194,0.067200,-0.897766,1.184709,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
552,28.305330,34.580994,0.992804,24.527611,3.663732,-10.643265,2.737070,1.422497,0.638672,0.62525,...,0.541381,0.302102,-0.010755,-0.01960,0.003617,0.060146,0.061100,0.473889,0.887487,0
553,27.689082,34.388985,0.991581,24.559595,3.724517,-10.598345,2.382061,1.260590,0.638003,0.62390,...,-0.256671,0.522196,-0.009470,0.00790,0.003039,0.055126,0.055934,-2.124340,5.710925,0
554,27.932589,35.284772,0.990221,24.578449,3.782127,-10.500461,2.506340,1.735253,0.637607,0.62390,...,0.541286,0.742225,0.006430,0.00675,0.002085,0.045667,0.046118,-0.517669,0.670844,0
555,26.978382,33.504319,0.991586,24.802881,4.305794,-9.484058,2.404188,1.539252,0.633807,0.62340,...,-0.321683,0.667451,-0.000010,0.00245,0.002417,0.049165,0.049165,-0.270820,0.446882,0


In [690]:
features.to_csv('jeongyubin.csv', index=False)

In [691]:
features2 = pd.read_csv('ex_all2.csv')

In [692]:
gg = pd.concat([features, features2])
gg

,A1D1_energy,A2D2_energy,WEE,orizinal_entropy,cA1_entropy,cA2_entropy,cD1_entropy,cD2_entropy,original_Mean,original_Median,...,cD1_Skewness,cD1_Kurtosis,cD2_Mean,cD2_Median,cD2_Variance,cD2_Std_deviation,cD2_RMS,cD2_Skewness,cD2_Kurtosis,label
0,27.686288,31.149462,0.997499,24.875515,4.908510,-8.088996,3.951457,1.838479,0.628808,0.62510,...,1.192917,3.837308,-0.005990,0.00245,0.004921,0.070151,0.070406,-0.756257,0.502008,2
1,25.228632,31.355733,0.991526,24.862751,4.854788,-8.247846,2.418375,1.816586,0.629080,0.62540,...,-1.380191,3.455281,-0.006875,-0.00470,0.006113,0.078183,0.078485,-0.365148,0.211409,2
2,27.916939,32.074378,0.996533,24.826024,4.821596,-8.344118,3.999195,1.990965,0.629827,0.62540,...,1.194736,3.890219,0.005100,0.00085,0.005647,0.075147,0.075320,0.944897,1.018330,2
3,25.258227,32.315169,0.989135,24.781981,4.782353,-8.494734,2.378887,1.875455,0.630710,0.62540,...,-1.390647,3.477314,0.005830,-0.00195,0.004833,0.069516,0.069760,-0.035377,1.011914,2
4,27.647997,31.909666,0.996303,24.761277,4.769470,-8.494714,3.522075,1.974006,0.631172,0.62650,...,1.400508,5.250005,-0.000870,0.00990,0.004515,0.067194,0.067200,-0.897766,1.184709,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
989,14.737896,18.363848,0.991327,30.406736,12.745386,1.565651,2.469990,2.446779,0.472530,0.46085,...,0.715656,2.680316,0.012595,0.01145,0.003963,0.062956,0.064203,-0.958372,1.444703,1
990,15.215484,17.739707,0.995764,30.417792,12.859324,1.857289,3.095051,1.994340,0.471967,0.46000,...,1.288816,2.849183,0.006545,-0.00150,0.004156,0.064468,0.064799,0.038491,-0.230645,1
991,14.169982,16.940546,0.994271,30.424605,12.884529,1.900349,2.402054,1.034922,0.471635,0.45930,...,-0.431458,0.303620,-0.013165,-0.01400,0.002623,0.051216,0.052881,0.383257,0.883143,1
992,15.302014,17.730795,0.996097,30.413893,12.777889,1.677272,2.904523,1.747434,0.472645,0.45930,...,1.444820,3.666208,-0.001345,0.00515,0.003876,0.062254,0.062269,0.105604,0.902611,1


In [693]:
gg.to_csv('ex_all3.csv', index=False)